## Метод модифицированной функции Лагранжа
#### Замятина Екатерина 476 группа

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('data1.csv', encoding='utf7').astype(float)

## Матрица ковариации

In [3]:
data.cov()

,SBER,MEGAFON,YANDEX,ROSN
SBER,0.001747,0.000317,0.001005,0.001107
MEGAFON,0.000317,0.001488,0.000438,0.000527
YANDEX,0.001005,0.000438,0.002834,0.000879
ROSN,0.001107,0.000527,0.000879,0.001927


In [2]:
import numpy as np
import math

In [5]:
t= 1
omega = np.array(data.cov())

$$min (\frac{1}{2} x^T \Omega x)$$

  где $\Omega$ - матрица ковариации и выполнены условия:


$$h(x) = \sum(x_i) - 1 = 0, g(x_i) = - x_i \le 0$$

$$L = \frac{1}{2} x^T \Omega x + \lambda(\sum(x_i) - 1) + \mu^T (-x_i)) + r(\sum(x_i) - 1)^2 + \sum(t_i (-x_i)^2_+), \mu_i \ge 0$$

Минимизируя по $\lambda$ и $\mu$ получаем метод:
$$ \lambda_{k+1} = \lambda_k + 2r(\sum(x_i(\mu_k,\lambda_k)) - 1), \mu_{k+1} = (\mu_k - 2rx(\mu_k, \lambda_k))_+ $$
$$x(\lambda, \mu) =  ( \mu - \lambda*1) \Omega^{-1}$$

Но эта задача у меня не получилась, поэтому решим сначала задачу попроше:
    $$min(2x_1^2 + 3x_2^2 - x_1 +x_1x_2)$$
    $$x_1+x_2=7$$
    $$L(x,\lambda) = (2x_1^2 + 3x_2^2 - x_1 +x_1x_2) + \lambda(x_1 +x_2 - 7)$$
    $$ Ax = t $$
    $$ A = \begin{pmatrix}
  4 & 1\\
  1& 6
\end{pmatrix} , t = - \lambda + \begin{pmatrix}
  1\\
  0
\end{pmatrix}$$

In [119]:
A = np.array([[4,1],[1,6]])

In [138]:
def f(x):
    x = np.array(x)
    return 2*x[0]*x[0] + 3*x[1]*x[1] - x[0] +x[0]*x[1]
    #return 0.5*np.dot(np.dot(x.T, omega), x)
def x(l): 
    #return np.dot(np.linalg.inv(omega),(m + l*np.ones(4)))
    t = np.array([[1],[0]])
    x = np.linalg.solve(A, -l +t)
    return x

def h(x):
    #return (np.sum(x) - 1)
    return np.sum(x) - 7

In [139]:
def method(t, start_lambda ,max_number_iterations, eps = 10**(-8)):
    current_lambda = start_lambda
    #current_mu = start_mu
    for i in range(max_number_iterations):
        current_x = x(current_lambda)       
        if (i > 0):
            
            if (((current_lambda - previous_lambda)*(current_lambda - previous_lambda) < eps*eps)): #and\
                #(((current_mu - previous_mu).dot(current_mu - previous_mu) < eps*eps))):
                return current_x, f(current_x), np.sum(current_x), i
            else:
                previous_lambda = current_lambda
                current_lambda = current_lambda + 2 * t * h(current_x) 
                #previous_mu = current_mu
                #current_mu = (current_mu - 2 * t * current_x).clip(min=0)
        else: 
            previous_lambda = current_lambda
            current_lambda = current_lambda + 2 * t * h(current_x)
            #previous_mu = current_mu
            #current_mu = (current_mu - 2 * t * current_x).clip(min=0)
            
                    
    return current_x, f(current_x), np.sum(current_x), "over 500000"

In [140]:
start_lambda = 0

In [143]:
df2 = pd.DataFrame({ 'Вес 0.05' : method(0.5,start_lambda, max_number_iterations=500000), 
                       'Вес 0.2' : method(0.2,start_lambda, max_number_iterations=500000),
                       'Вес 0.25' : method(0.25,start_lambda, max_number_iterations=500000),
                    }, index = ["x", "f(x)", "sum(x)", "method converged at iteration"])

In [116]:
df2

,Вес 0.05,Вес 0.2,Вес 0.25
x,"[[4.49999994102], [2.49999996461]]","[[4.49999998729], [2.49999999238]]","[[4.49999999005], [2.49999999403]]"
f(x),[65.9999981597],[65.9999996035],[65.9999996895]
sum(x),7,7,7
method converged at iteration,511,131,104


#### Аналитическое решение

$f(x) = 2x_1^2 + 3x_2^2 - x_1 +x_1x_2 + t(x_1 + x_2 - 7)^2$ - выпуклая функция, поэтому $\nabla f(x) = 0$ необходимое и достаточное условие экстремума. 
$$\nabla f(x) = Ax + 2t(x_1 + x_2 -7)*1 - b = 0$$
$$ Bx = 14t*1 + b$$
$$ B = \begin{pmatrix}
  4 + 2t& 1 + 2t\\
  1 + 2t& 6 + 2t
\end{pmatrix}, b = \begin{pmatrix}
  1\\
  0
\end{pmatrix}$$

In [135]:
t = 20
B = A + 2*t*(np.ones(2)*np.ones(2).T)
x = np.linalg.solve(B, 14*t*np.ones(2) + [1,0])

In [136]:
x

array([ 4.21574344,  2.32944606])

## Конец